In [12]:
import numpy as np
import pandas as pd
import cv2
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [14]:
# Load dataset
df = pd.read_csv("WildFires_DataSet.csv")

# Split features & labels
X = df[['NDVI', 'LST', 'BURNED_AREA']].values
y = df['CLASS'].map({'no_fire': 0, 'fire': 1}).values


In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [16]:
# Ignore

IMG_SIZE = 224

def row_to_image(row):
    # row is scaled NDVI, LST, Thermal
    ndvi, lst, thermal = row

    # Normalize values 0-255
    ndvi_norm = int((ndvi - X_scaled[:,0].min()) / (X_scaled[:,0].max() - X_scaled[:,0].min()) * 255)
    lst_norm = int((lst - X_scaled[:,1].min()) / (X_scaled[:,1].max() - X_scaled[:,1].min()) * 255)
    thermal_norm = int((thermal - X_scaled[:,2].min()) / (X_scaled[:,2].max() - X_scaled[:,2].min()) * 255)

    # Create plain 224x224 RGB image
    img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
    img[:, :, 0] = ndvi_norm
    img[:, :, 1] = lst_norm
    img[:, :, 2] = thermal_norm

    return img

# Convert all rows to images
images = np.array([row_to_image(row) for row in X_scaled])
print(images.shape)


(1713, 224, 224, 3)


In [18]:
IMG_SIZE = 224

def row_to_pattern(row):
    ndvi, lst, burn = row

    img = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)

    # Gaussian patterns for each channel
    x = np.linspace(-1, 1, IMG_SIZE)
    y = np.linspace(-1, 1, IMG_SIZE)
    xx, yy = np.meshgrid(x, y)

    # Channel 1 — NDVI
    pattern1 = np.exp(-(xx**2 + yy**2) * (4 + ndvi*6))

    # Channel 2 — LST
    pattern2 = np.exp(-(xx**2 + yy**2) * (4 + (lst)*2))

    # Channel 3 — Burned_area
    pattern3 = np.exp(-(xx**2 + yy**2) * (4 + burn*5))

    img[:,:,0] = (pattern1 / pattern1.max() * 255).astype(np.uint8)
    img[:,:,1] = (pattern2 / pattern2.max() * 255).astype(np.uint8)
    img[:,:,2] = (pattern3 / pattern3.max() * 255).astype(np.uint8)

    return img

images = np.array([row_to_pattern(row) for row in X_scaled])
print(images.shape)


(1713, 224, 224, 3)


In [19]:
X_train, X_test, y_train, y_test = train_test_split(
    images,
    y,
    test_size=0.25,
    random_state=42,
    stratify=y
)


In [8]:
# Load ResNet50 without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze all layers except last few
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    metrics=['accuracy']
)

model.summary()


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,850,113 (90.98 MB)

 Trainable params: 4,728,065 (18.04 MB)

 Non-trainable params: 19,122,048 (72.94 MB)

In [9]:
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    callbacks=[early_stop]
)


Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 39s 668ms/step - accuracy: 0.7674 - loss: 0.5321 - val_accuracy: 0.7708 - val_loss: 0.5139
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 101ms/step - accuracy: 0.7782 - loss: 0.4598 - val_accuracy: 0.7542 - val_loss: 0.5178
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - accuracy: 0.7772 - loss: 0.4640 - val_accuracy: 0.7625 - val_loss: 0.5104
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 107ms/step - accuracy: 0.7906 - loss: 0.4458 - val_accuracy: 0.7750 - val_loss: 0.5029
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - accuracy: 0.8003 - loss: 0.4340 - val_accuracy: 0.7583 - val_loss: 0.4934
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 106ms/step - accuracy: 0.8124 - loss: 0.4054 - val_accuracy: 0.7500 - val_loss: 0.4899
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 116ms/step - accuracy: 0.8187 - loss: 0.4344 - val_accuracy: 0.7583 - val_loss: 0.4942
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 102ms/step - accuracy: 0.8184 - loss: 0.4125 - val_accuracy: 0

In [10]:
loss, accuracy = model.evaluate(X_test, y_test)
print("🔥 Test Accuracy:", accuracy * 100)


17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 273ms/step - accuracy: 0.8085 - loss: 0.4353
🔥 Test Accuracy: 78.21011543273926
